### 1) Copy and Rename MCD older from old version-->new version


In [ ]:
import shutil
import os
import arcpy

# === Set version info ===
old_version = "12"
new_version = "13"

# === Define paths ===
base_path = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\_MCD"
folder_template = "005356185-{} - THO - Master Construction Drawing_20250724"


src_folder = os.path.join(base_path, folder_template.format(old_version))
dst_folder = os.path.join(base_path, folder_template.format(new_version))

# === Step 1: Copy the folder ===
try:
    shutil.copytree(src_folder, dst_folder)
    print(f"✅ Folder copied from:\n{src_folder}\nto:\n{dst_folder}")
except FileExistsError:
    print(f"⚠️ Destination folder already exists:\n{dst_folder}")
 
except FileNotFoundError:
    print(f"❌ Source folder not found:\n{src_folder}")
  
except Exception as e:
    print(f"❌ Error copying folder:\n{e}")


# Rename files inside the copied folder ===
renamed_files = 0
for root, dirs, files in os.walk(dst_folder):
    for filename in files:
        print(filename)
        # Skip if the file already contains the new version
        if f"-{new_version}" in filename:
            continue
        # Proceed if the file contains the old version
        if f"-{old_version}" in filename:
            old_path = os.path.join(root, filename)
            new_filename = filename.replace(f"-{old_version}", f"-{new_version}")
            new_path = os.path.join(root, new_filename)
            os.rename(old_path, new_path)
            renamed_files += 1
            print(f"🔁 Renamed: {filename} → {new_filename}")

print(f"✅ Finished. {renamed_files} files renamed in folder version {new_version}.")

### 3) As Built Monopiles

Please complete all steps **before running the script below**:

1. 🔽 **Download PDFs**  
   - URL: [https://share.jandenul.com/](https://share.jandenul.com/)  
   - **Username**: DKTRTI,  **Password**: RKHJEB5LvcnVZw2

2. 💾 **Save PDFs** here:  
   *\\WM20ocqu46ph01\WF_Projects\DK_THO\1_INPUT\CONSTR\2025\20250505 As Built Monopiles from JDN*


3. 📋 **Extract and copy relevant data** into this Excel file:  
   *\\WM20ocqu46ph01\WF_Projects\DK_THO\1_INPUT\CONSTR\2025\20250505 As Built Monopiles from JDN\DK_THO_AsBuilt_Monopile_pt_UTM32N_ta_v0.xlsx*


4. 📤 **Export the cleaned data to CSV**: 
   *\\WM20ocqu46ph01\WF_Projects\DK_THO\1_INPUT\CONSTR\2025\20250505 As Built Monopiles from JDN\DK_THO_AsBuilt_Monopile_pt_UTM32N_ta_v0.csv*
   
❗ **Ensure that the `005356185_THOR_Master_Construction_Drawing.aprx` project is closed** before running the script — otherwise layer will be not in the content (run over different aprx or VS code) 

              Two files will be final ouputs saved in WTG.gdb:
              - *DK_THO_AsBuilt_Monopile_pt_UTM32N_pt_v0*
              - *DK_THO_AsBuilt_Monopile_OuterDiameter8pt6m_py_UTM32N_pt_v0*
              - *DK_THO_AsBuilt_Monopile_pt_UTM32N_pt_v0_Annotation*
5. ✅ Run the code



#####  📌AS BUILT - Monopile Location

Change in the code: `new_version = "XX"`

In [ ]:
import shutil
import os
import arcpy

new_version = "13"

# === INPUTS ===
csv_file = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\1_INPUT\CONSTR\2025\20250505 As Built Monopiles from JDN\DK_THO_AsBuilt_Monopile_pt_UTM32N_ta_v0.csv"
x_field = "POS_ASBUILT_EAST"
y_field = "POS_ASBUILT_NORTH"
spatial_ref = arcpy.SpatialReference(32632)  # UTM Zone 32N

# === OUTPUTS ===
point_fc = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\2_FINAL\WTG\WTG.gdb\DK_THO_AsBuilt_Monopile_pt_UTM32N_pt_v0"
buffer_fc = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\2_FINAL\WTG\WTG.gdb\DK_THO_AsBuilt_Monopile_OuterDiameter8pt6m_py_UTM32N_pt_v0"


# === STEP 1: Create Point Feature Class ===
if arcpy.Exists(point_fc):
    arcpy.Delete_management(point_fc)
    print(f"✅ Point feature class was deleted: {point_fc}")

arcpy.management.XYTableToPoint(
    in_table=csv_file,
    out_feature_class=point_fc,
    x_field=x_field,
    y_field=y_field,
    coordinate_system=spatial_ref
)

print(f"✅ Point feature class created: {point_fc}")

# === STEP 2: Create 4.3m Buffer ===
if arcpy.Exists(buffer_fc):
    arcpy.Delete_management(buffer_fc)
    print(f"✅ Point feature class was deleted: {buffer_fc}")

arcpy.analysis.PairwiseBuffer(
    in_features=point_fc,
    out_feature_class=buffer_fc,
    buffer_distance_or_field="4.3 Meters",
    dissolve_option="NONE"
)

with arcpy.da.UpdateCursor(buffer_fc, ['LAYER']) as cursor:
    for row in cursor:
        row[0] = 'AS BUILT - Monopile Outer Diameter 8pt6m'
        cursor.updateRow(row)

print(f"✅ Buffer feature class created: {buffer_fc}")


#####  📌 ANNOTATION – As Built Monopile Location - {8-12min}

Change in the code: `new_version = "XX"`

In [ ]:
# === Load the project ===
import arcpy

new_version = "13"

# === INPUTS ===
aprx_path = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\_MCD\005356185_THOR_Master_Construction_Drawing.aprx"
output_gdb = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\2_FINAL\WTG\WTG.gdb"
layer_name = "As Built Monopile"
output_annotation_suffix = "_Annotation"
new_map_name = f"005356185-{new_version} THOR Master Construction Drawing CAD"
reference_scale = 100

aprx = arcpy.mp.ArcGISProject(aprx_path)
target_map = next((m for m in aprx.listMaps() if m.name == new_map_name), None)
if not target_map:
    raise Exception(f"❌ Map '{new_map_name}' not found.")
print(f"✅ Map '{new_map_name}' loaded.")

# === Find the layer ===
layer = next((lyr for lyr in target_map.listLayers() if lyr.name == layer_name), None)
if not layer or not layer.supports("SHOWLABELS"):
    raise Exception(f"❌ Layer '{layer_name}' not found or doesn't support labels.")

layer.showLabels = True
print(f"✅ Labels enabled for '{layer_name}'.")


arcpy.cartography.ConvertLabelsToAnnotation(
    input_map= target_map,
    conversion_scale= 100,
    output_geodatabase= output_gdb,
    anno_suffix= output_annotation_suffix,
    extent='394631.907089481 6223623.59608526 450316.163918058 6267404.33424396 PROJCS["WGS_1984_UTM_Zone_32N",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",9.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]',
    generate_unplaced="GENERATE_UNPLACED",
    require_symbol_id=None,
    feature_linked=None,
    auto_create="AUTO_CREATE",
    update_on_shape_change="SHAPE_UPDATE",
    output_group_layer="GroupAnno",
    which_layers="SINGLE_LAYER",
    single_layer=layer,
    multiple_feature_classes="FEATURE_CLASS_PER_FEATURE_LAYER",
    merge_label_classes="NO_MERGE_LABEL_CLASS"
)


# Rename the feature class, if the n
annotation_fc = f"{output_gdb}/As_Built_Monopile{output_annotation_suffix}"
new_ann_fc = f"{output_gdb}/DK_THO_AsBuilt_Monopile_pt_UTM32N_pt_v0_Annotation"     

if arcpy.Exists(annotation_fc):
    print(f"✅ Annotation feature class saved at: {annotation_fc}")
    if arcpy.Exists(new_ann_fc):
        arcpy.Delete_management(new_ann_fc)
    arcpy.Rename_management(annotation_fc, new_ann_fc)
    print(f"✅ Renamed annotation feature class to: {new_ann_fc}")
else:
    # Print warning if not found
    print(f"⚠️ Annotation feature class NOT found at: {annotation_fc}")
    
    

# Check if feature class exists
if arcpy.Exists(new_ann_fc):
    # Add 'Layer' field if it doesn't exist
    if "Layer" not in [f.name for f in arcpy.ListFields(new_ann_fc)]:
        arcpy.AddField_management(new_ann_fc, "Layer", "TEXT", field_length=255)
        print(f"➕ Added missing 'Layer' field to: {new_ann_fc}")
    else:
        print(f"ℹ️ 'Layer' field already exists in: {new_ann_fc}")
    count = 0
    with arcpy.da.UpdateCursor(new_ann_fc, ["Layer"]) as cursor:
        for row in cursor:
            if row[0] != "ANNOTATION - As Built Monopile Location":
                row[0] = "ANNOTATION - As Built Monopile Location"
                cursor.updateRow(row)
                count += 1
else:
    print(f"⚠️ Feature class not found: {new_ann_fc}")
    
print("✅✅ Layer field values updated **ANNOTATION - As Built Monopile Location** successfully.")    
    

### 4) VESSEL - As Built Footprint - MPI Adventure

1) Open **EcoCod** and search for the title: *'VAO - Adventure T*'*
2) The relevant file from ecodoc should be saved to: `\\WM20ocqu46ph01\WF_Projects\DK_THO\1_INPUT\CONSTR\2025\20250604 As Built Secondary Steel Installation from Van Oord`
3) Run Code


  ##### 📌VESSEL - As Built Footprint - MPI  Adventure (Polygon and Annotation)

In [ ]:
import os
import arcpy
import re
import pandas as pd

# === CONFIGURATION ===
ids = [f"{i:02}" for i in range(0, 73)]
name_polygon = "DK_THO_MPIAdventure_AsBuiltFootprints_py_UTM32N_v1"
name_ann = "DK_THO_MPIAdventure_AsBuiltFootprints_Annotation_UTM32N_v1"

spatial_ref = arcpy.SpatialReference(32632)

input_dir = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\1_INPUT\CONSTR\2025\20250604 As Built Secondary Steel Installation from Van Oord"
output_gdb = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\2_FINAL\VESSELS\VESSELS.gdb"

final_fc_polygon = os.path.join(output_gdb, name_polygon)
final_fc_ann = os.path.join(output_gdb, name_ann)

export_fc_py = []
export_fc_ann= []

fields_to_add = [("LAYER", "TEXT"), ("SOURCE", "TEXT"),  ("NEAR_IF", "TEXT"),("WTG_ID", "TEXT")]

def sanitize_name(name):
    name = os.path.splitext(name)[0]
    name = re.sub(r'[^A-Za-z0-9_]', '_', name)
    name = re.sub(r'__+', '_', name)
    if not re.match(r'^[A-Za-z]', name):
        name = "D_" + name
    return name[:63]

# === Process each turbine ID ===
for id_ in ids:
    wtg_id = f"T{id_}"
    turbine_folder = os.path.join(input_dir, wtg_id)
    for root, dirs, files in os.walk(turbine_folder):
        for file in files:
            if file.lower().endswith(".dwg"):
                dwg_path = os.path.join(root, file)
                print(f"✅ {wtg_id} - Processing")

                processing_path = os.path.join(turbine_folder, "PROCESSING")
                os.makedirs(processing_path, exist_ok=True)
                gdb_path = os.path.join(processing_path, f"DWG_{wtg_id}.gdb")

                # arcpy.Delete_management(gdb_path)
                if not arcpy.Exists(gdb_path):
                    arcpy.CreateFileGDB_management(processing_path, f"DWG_{wtg_id}.gdb")
                    name_dwg = sanitize_name(file)
                    arcpy.conversion.CADToGeodatabase(dwg_path, gdb_path, name_dwg, "1000", "")
                    print(f"✅ {wtg_id} - DWG converted to GDB: {gdb_path}")                 

                fc_Polygon = os.path.join(gdb_path, "Polygon")    
                if not arcpy.Exists(fc_Polygon):
                    print(f"❌ {wtg_id} - Polygon feature class not found in GDB.")
                    continue
             
                # Cleanup fields
                fields_fc_Polygon = arcpy.ListFields(fc_Polygon)
                fields_to_delete = [f.name for f in fields_fc_Polygon if f.name not in ("OBJECTID", "SHAPE") and not f.required]
                if fields_to_delete:
                    arcpy.DeleteField_management(fc_Polygon, fields_to_delete)
                    # print(f"✅ {wtg_id} - Fields are deleting")
                else:
                    print(f"{wtg_id} - No deletable fields in {fc_Polygon}")

                for field_name, ftype in fields_to_add:    
                    if field_name not in arcpy.ListFields(fc_Polygon):
                        arcpy.AddField_management(fc_Polygon, field_name, ftype)   
                # print(f"✅ {wtg_id} - Adding fields")
                with arcpy.da.UpdateCursor(fc_Polygon, ["LAYER", "SOURCE","NEAR_IF","WTG_ID"]) as cursor:
                    for row in cursor:
                        row[0] = "VESSEL - As Built Footprint - MPI Adventure"
                        row[1] = "Van Oord / MPI Adventure"  
                        row[2] =  id_                    
                        row[3] = 'T'+id_
                        cursor.updateRow(row)
                export_fc_py.append(fc_Polygon)


                fc_Annotation = os.path.join(gdb_path, "Annotation")
                if not arcpy.Exists(fc_Annotation):
                    print(f"❌ {wtg_id} - Annotation feature class not found in GDB.")
                    continue
                for field_name, ftype in fields_to_add:    
                    if field_name not in arcpy.ListFields(fc_Annotation):
                        arcpy.AddField_management(fc_Annotation, field_name, ftype)
                with arcpy.da.UpdateCursor(fc_Annotation, ["LAYER", "SOURCE","NEAR_IF","WTG_ID"]) as cursor:
                    for row in cursor:
                        row[0] = "ANNOTATION - As Built Footprint - MPI Adventure"
                        row[1] = "Van Oord / MPI Adventure"  
                        row[2] =  id_                    
                        row[3] = 'T'+id_
                        cursor.updateRow(row)
                export_fc_ann.append(fc_Annotation)

# Annotation
if arcpy.Exists(final_fc_ann):
    arcpy.Delete_management(final_fc_ann) 
arcpy.Merge_management(inputs=export_fc_ann, output=final_fc_ann)
arcpy.DefineProjection_management(final_fc_ann, spatial_ref)
print(f" ✅ Total Annotation {len(export_fc_ann)} IDs are merged in: {final_fc_ann}")


# Polygon        
if arcpy.Exists(final_fc_polygon):
    arcpy.Delete_management(final_fc_polygon) 
arcpy.Merge_management(inputs=export_fc_py, output=final_fc_polygon,)
arcpy.DefineProjection_management(final_fc_polygon, spatial_ref)
print(f" ✅ Total {len(export_fc_py)} IDs are merged in: {final_fc_polygon}")

# Final summary
fields = [f.name for f in arcpy.ListFields(final_fc_polygon) if f.name and f.type not in ('Geometry', 'OID')]
arr = arcpy.da.FeatureClassToNumPyArray(final_fc_polygon, fields)
df = pd.DataFrame(arr)
print(f"✅ Loaded positions: {int(len(df) / 6)}\nWTG_IDs: {df['WTG_ID'].unique()}")


  
  #####  📌VESSEL - As Built Footprint - MPI  Adventure (Point)

In [3]:
import os
import arcpy
import re
import pandas as pd

# === CONFIGURATION ===
# ids = ['01']
# ids = ['01','64', '35','71','58','25','60','29']
ids = [f"{i:02}" for i in range(1, 73)]
name_pt = "DK_THO_MPIAdventure_AsBuiltFootprints_pt_UTM32N_v1"

input_dir = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\1_INPUT\CONSTR\2025\20250604 As Built Secondary Steel Installation from Van Oord"
output_gdb = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\2_FINAL\VESSELS\VESSELS.gdb"

spatial_ref = arcpy.SpatialReference(32632)

final_fc_point = os.path.join(output_gdb, name_pt)
export_fcs_pt = []

fields_to_add = [("LAYER", "TEXT"), ("SOURCE", "TEXT"),  ("NEAR_IF", "TEXT"),("WTG_ID", "TEXT")]

def sanitize_name(name):
    name = os.path.splitext(name)[0]
    name = re.sub(r'[^A-Za-z0-9_]', '_', name)
    name = re.sub(r'__+', '_', name)
    if not re.match(r'^[A-Za-z]', name):
        name = "D_" + name
    return name[:63]

# === Process each turbine ID ===
for id_ in ids:
    wtg_id = f"T{id_}"
    turbine_folder = os.path.join(input_dir, wtg_id)
    for root, dirs, files in os.walk(turbine_folder):
        for file in files:
            if file.lower().endswith(".dwg"):
                dwg_path = os.path.join(root, file)
                print(f"✅ {wtg_id} - Processing")

                processing_path = os.path.join(turbine_folder, "PROCESSING")
                os.makedirs(processing_path, exist_ok=True)
                gdb_path = os.path.join(processing_path, f"DWG_{wtg_id}.gdb")
                
                #arcpy.Delete_management(gdb_path)
                if not arcpy.Exists(gdb_path):
                    arcpy.CreateFileGDB_management(processing_path, f"DWG_{wtg_id}.gdb")
                    name_dwg = sanitize_name(file)
                    arcpy.conversion.CADToGeodatabase(dwg_path, gdb_path, name_dwg, "1000", "")
                    print(f"✅ {wtg_id} - DWG converted to GDB")                 

                fc = os.path.join(gdb_path, "Point")
                if not arcpy.Exists(fc):
                    print(f"❌ {wtg_id} - Point feature class not found in GDB.")
                    continue
              
                # Cleanup fields
                fields_in_fcs = arcpy.ListFields(fc)

                fields_to_delete = [f.name for f in fields_in_fcs if f.name not in ("OBJECTID", "SHAPE", "RefName", "Angle", "LEG_PEN1", "VESSEL1", "DATE1") and not f.required]

                if fields_to_delete:
                    arcpy.DeleteField_management(fc, fields_to_delete)
                    # print(f"✅ {wtg_id} - Fields are deleting")
                else:
                    print(f"{wtg_id} - No deletable fields in {fc}")

                for field_name, ftype in fields_to_add:    
                    if field_name not in arcpy.ListFields(fc):
                        arcpy.AddField_management(fc, field_name, ftype)
                    
                # print(f"✅ {wtg_id} - Adding fields")

                with arcpy.da.UpdateCursor(fc, ["LAYER", "SOURCE","NEAR_IF","WTG_ID"]) as cursor:
                    for row in cursor:
                        row[0] = "VESSEL - As Built Footprint - MPI Adventure Point"
                        row[1] = "Van Oord / MPI Adventure"  
                        row[2] =  id_                    
                        row[3] = 'T'+id_
                        cursor.updateRow(row)

                                # Fields to update
                fields = ["LEG_PEN1", "VESSEL1", "DATE1"]

                # Cleanup temp layers
                has_vals_name = f"has_vals_{wtg_id}"
                null_vals_name = f"null_vals_{wtg_id}"
                near_tbl = f"in_memory/near_{wtg_id}"

                # Clean up if they already exist
                for temp in [has_vals_name, null_vals_name, near_tbl]:
                    if arcpy.Exists(temp):
                        arcpy.Delete_management(temp)

                # Make layers
                has_vals = arcpy.MakeFeatureLayer_management(fc, has_vals_name, " AND ".join([f"{f} IS NOT NULL" for f in fields]))
                null_vals = arcpy.MakeFeatureLayer_management(fc, null_vals_name, " OR ".join([f"{f} IS NULL" for f in fields]))

                # Generate near table
                arcpy.analysis.GenerateNearTable(null_vals, has_vals, near_tbl, closest=True)

                # Create lookups
                val_lookup = {r[0]: r[1:] for r in arcpy.da.SearchCursor(has_vals, ["OID@"] + fields)}
                near_map = {r[0]: r[1] for r in arcpy.da.SearchCursor(near_tbl, ["IN_FID", "NEAR_FID"])}

                # Update nulls using nearest values
                with arcpy.da.UpdateCursor(null_vals, ["OID@"] + fields) as cursor:
                    for row in cursor:
                        donor = val_lookup.get(near_map.get(row[0]))
                        if donor:
                            row[1:] = [donor[i] if not row[i+1] else row[i+1] for i in range(len(fields))]
                            cursor.updateRow(row)

                export_fcs_pt.append(fc)
        
if arcpy.Exists(final_fc_point):
    arcpy.Delete_management(final_fc_point) 

arcpy.Merge_management(inputs=export_fcs_pt, output=final_fc_point)
arcpy.DefineProjection_management(final_fc_point, spatial_ref)

# Delete features where Angle = 0
if "Angle" in [f.name for f in arcpy.ListFields(final_fc_point)]:
    layer_name = "in_memory\\angle_layer"
    arcpy.management.MakeFeatureLayer(final_fc_point, layer_name)
    arcpy.management.SelectLayerByAttribute(layer_name, "NEW_SELECTION", '"Angle" = 0')
    arcpy.management.DeleteFeatures(layer_name)

# Final summary
fields = [f.name for f in arcpy.ListFields(final_fc_point) if f.name and f.type not in ('Geometry', 'OID')]
arr = arcpy.da.FeatureClassToNumPyArray(final_fc_point, fields)
df = pd.DataFrame(arr)
print(f"✅ Loaded positions: {int(len(df) / 6)}\nWTG_IDs: {df['WTG_ID'].unique()}")


✅ T20 - Processing
✅ T23 - Processing
✅ T24 - Processing
✅ T25 - Processing
✅ T26 - Processing
✅ T28 - Processing
✅ T29 - Processing
✅ T34 - Processing
✅ T35 - Processing
✅ T42 - Processing
✅ T43 - Processing
✅ T53 - Processing
✅ T54 - Processing
✅ T55 - Processing
✅ T58 - Processing
✅ T59 - Processing
✅ T60 - Processing
✅ T61 - Processing
✅ T64 - Processing
✅ T68 - Processing
✅ T69 - Processing
✅ T70 - Processing
✅ T71 - Processing
✅ Loaded positions: 34
WTG_IDs: ['T01' 'T02' 'T03' 'T04' 'T05' 'T06' 'T09' 'T14' 'T15' 'T18' 'T19' 'T20'
 'T23' 'T24' 'T25' 'T26' 'T28' 'T29' 'T34' 'T35' 'T42' 'T43' 'T53' 'T54'
 'T55' 'T58' 'T59' 'T60' 'T61' 'T64' 'T68' 'T69' 'T70' 'T71']


  #####  📌VESSEL - Ghost Position Sector - MPI Adventure - VOOW - LEGS_PRESENT Field Update
 

 This script performs a spatial join to identify which sector polygons contain MPI Adventure footprint points, then updates the LEGS_PRESENT field in the sector feature class to "Yes" for those sectors that contain at least one point.

🛠️ LEGS_PRESENT field in the **sector feature class** (DK_THO_MPIAdventure_VesselGhostPositioningSectors_py_UTM32N_v2) has the query:
- 'LEGS_PRESENT = NO':
  - ✅ `LEGS_PRESENT = 'Yes'` — if the sector contains **MPI footprint point**  
  -  ❌ `LEGS_PRESENT = 'No'` — if the sector contains **no points**

In [4]:
import arcpy
import pandas as pd

# Inputs
points = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\2_FINAL\VESSELS\VESSELS.gdb\DK_THO_MPIAdventure_AsBuiltFootprints_pt_UTM32N_v1"
sectors = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\2_FINAL\VESSELS\VESSELS.gdb\DK_THO_MPIAdventure_VesselGhostPositioningSectors_py_UTM32N_v2"
update_field = "LEGS_PRESENT"

# Temporary in-memory join result
intersect_layer = "in_memory\\point_sector_intersect"

# Delete if already exists
if arcpy.Exists(intersect_layer):
    arcpy.Delete_management(intersect_layer)

# Perform spatial join
arcpy.analysis.SpatialJoin(
    target_features=sectors,
    join_features=points,
    out_feature_class=intersect_layer,
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_ALL",
    match_option="INTERSECT"
)

# Build a set of sectors that have Join_Count > 0
intersect_ids = set()
with arcpy.da.SearchCursor(intersect_layer, ["TARGET_FID", "Join_Count"]) as cursor:
    for target_fid, join_count in cursor:
        if join_count > 0:
            intersect_ids.add(target_fid)

# Debug: print counts
print(f"Sectors with points (footprints of sst): {len(intersect_ids)}")

# Update sectors accordingly
with arcpy.da.UpdateCursor(sectors, ["OBJECTID", update_field]) as cursor:
    for row in cursor:
        row[1] = "Yes" if row[0] in intersect_ids else "No"
        cursor.updateRow(row)

# Step 2: Read WTGID and LEGS_PRESENT for all sectors
wtgid_field = "WTGID"
fields = [wtgid_field, update_field]

wtgid_status = {}

with arcpy.da.SearchCursor(sectors, fields) as cursor:
    for wtgid, legs_present in cursor:
        if wtgid not in wtgid_status:
            wtgid_status[wtgid] = False
        if legs_present == "Yes":
            wtgid_status[wtgid] = True  # Mark this WTGID as having at least one "Yes"

# Step 3: Update all sectors based on group status
with arcpy.da.UpdateCursor(sectors, [wtgid_field, update_field]) as cursor:
    for wtgid, legs_present in cursor:
        new_value = "Yes" if wtgid_status.get(wtgid, False) else "No"
        if legs_present != new_value:
            cursor.updateRow([wtgid, new_value])


# Load updated sectors to numpy array and then pandas DataFrame
fields = [f.name for f in arcpy.ListFields(sectors) if f.name and f.type not in ('Geometry', 'OID')]
arr = arcpy.da.FeatureClassToNumPyArray(sectors, fields)
df = pd.DataFrame(arr)

# Print count summary of Yes and No
summary = df['LEGS_PRESENT'].value_counts(dropna=False).rename_axis('LEGS_PRESENT').reset_index(name='Count')
print(summary)


Sectors with points (footprints of sst): 34
  LEGS_PRESENT  Count
0           No     67
1          Yes     58


### 5) Armour Layer - Raster Mosaic Dataset

1. Ask Dele to get a SHP file from JDN and save them to:` \\WM20ocqu46ph01\WF_Projects\DK_THO\1_INPUT\CONSTR\2025\20250725 As Built Armour Layer Installation From JDN\Outlines\Outlines`
2. Run Code

##### 📌AS BUILT - Armour Layer Extent

In [2]:
import arcpy
import os
import re

# Set workspace and folders
root_folder = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\1_INPUT\CONSTR\2025\20250725 As Built Armour Layer Installation From JDN\Outlines\Outlines"
final_gdb = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\2_FINAL\CONSTR\CONSTR.gdb"
arcpy.env.workspace = root_folder
arcpy.env.overwriteOutput = True

shapefiles_to_merge = []

# Loop through directories and find all shapefiles
for dirpath, dirnames, filenames in os.walk(root_folder):
    for filename in filenames:
        if filename.lower().endswith(".shp"):
            shp_path = os.path.join(dirpath, filename)

            # Extract TXX from filename using regex
            match = re.search(r'(T\d{2})', filename)
            wtg_value = match.group(1) if match else "UNKNOWN"
            print(f"Processing shapefile: {filename} with WTG = {wtg_value}")

            # Add WTG field if it does not exist
            field_names = [f.name for f in arcpy.ListFields(shp_path)]
            if "WTG" not in field_names:
                arcpy.AddField_management(shp_path, "WTG", "TEXT", field_length=10)

            # Populate WTG field
            with arcpy.da.UpdateCursor(shp_path, ["WTG"]) as cursor:
                for row in cursor:
                    row[0] = wtg_value
                    cursor.updateRow(row)

            shapefiles_to_merge.append(shp_path)

# Output merged feature class
output_ArmourLayer = os.path.join(final_gdb, "DK_THO_ArmourLayer_AsBuiltExtent_py_UTM32N_v0")

# Merge all shapefiles
if shapefiles_to_merge:
    arcpy.Merge_management(shapefiles_to_merge, output_ArmourLayer)
    print(f"✅ Successfully merged into: {output_ArmourLayer}")

    # Add LAYER, SOURSE, DOC fields to the merged feature class
    fields_to_add = [
        ("LAYER", "TEXT", "CONSTRUCTION - Armour Layer As Built Extent"),
        ("SOURCE", "TEXT", "JDN"),
        ("DOC", "TEXT", root_folder)
    ]

    existing_fields = [f.name for f in arcpy.ListFields(output_ArmourLayer)]

    for field_name, field_type, _ in fields_to_add:
        if field_name not in existing_fields:
            arcpy.AddField_management(output_ArmourLayer, field_name, field_type, field_length=255)

    # Update the values in the new fields
    with arcpy.da.UpdateCursor(output_ArmourLayer, [f[0] for f in fields_to_add]) as cursor:
        for row in cursor:
            for i, (_, _, value) in enumerate(fields_to_add):
                row[i] = value
            cursor.updateRow(row)

else:
    print("⚠️ No shapefiles found to merge.")


Processing shapefile: JDN7723-SUR-BI-PST_AR_CONTOURS-T08.shp with WTG = T08
Processing shapefile: JDN7723-SUR-BI-PST_AR_CONTOURS-T10.shp with WTG = T10
Processing shapefile: JDN7723-SUR-BI-PST_AR_CONTOURS-T11.shp with WTG = T11
Processing shapefile: JDN7723-SUR-BI-PST_AR_CONTOURS-T12.shp with WTG = T12
Processing shapefile: JDN7723-SUR-BI-PST_AR_CONTOURS-T17.shp with WTG = T17
Processing shapefile: JDN7723-SUR-BI-PST_AR_CONTOURS-T22.shp with WTG = T22
Processing shapefile: JDN7723-SUR-BI-PST_AR_CONTOURS-T27.shp with WTG = T27
Processing shapefile: JDN7723-SUR-BI-PST_AR_CONTOURS-T30.shp with WTG = T30
Processing shapefile: JDN7723-SUR-BI-PST_AR_CONTOURS-T32.shp with WTG = T32
Processing shapefile: JDN7723-SUR-BI-PST_AR_CONTOURS-T33.shp with WTG = T33
Processing shapefile: JDN7723-SUR-BI-PST_AR_CONTOURS-T36.shp with WTG = T36
Processing shapefile: JDN7723-SUR-BI-PST_AR_CONTOURS-T45.shp with WTG = T45
Processing shapefile: JDN7723-SUR-BI-PST_AR_CONTOURS-T47.shp with WTG = T47
Processing s

##### 📌ARMOUR RASTER MOSAIC DATASET

1) [Download Raster data from this SharePoint folder](https://rwe.sharepoint.com/teams/ThorScourProtectionOffshoreExecution/Shared%20Documents/Forms/AllItems.aspx?csf=1&web=1&e=9bvbjG&FolderCTID=0x01200039B049B06B30784BAE472FE9C6E0D5FD&id=%2Fteams%2FThorScourProtectionOffshoreExecution%2FShared%20Documents%2FScour%20Protection%20Offshore%20Execution%2FLocation%20In%20Out%20Surveys%2F02%20Post%2Dsurveys)
2. Save them to:`\\WM20ocqu46ph01\WF_Projects\DK_THO\1_INPUT\CONSTR\2025\20250725 As Built Armour Layer Installation From JDN\OneDrive_2025-07-28\02 Post-surveys`

- Information:[Combining Mosaics into a single Image Service](https://rwe-renewables.atlassian.net/wiki/spaces/GISPORTAL/pages/636782242/Combining+Mosaics+into+a+single+Image+Service)

In [4]:
import os
import arcpy

# --- USER CONFIGURATION ---
base_dir = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\1_INPUT\CONSTR\2025\20250725 As Built Armour Layer Installation From JDN\OneDrive_2025-07-28\02 Post-surveys"
final_gdb = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\2_FINAL\CONSTR\CONSTR.gdb"

mosaic_name = "DK_THO_AsBuilt_ArmourLayer_ra_UTM32N_v0"
product_name_PRE = "DK_THO_AsBuilt_ArmourLayer_PreSurvey_ra_UTM32N_v0"
product_name_PST = "DK_THO_AsBuilt_ArmourLayer_PostSurvey_ra_UTM32N_v0"
spatial_ref = arcpy.SpatialReference(32632)

arcpy.env.workspace = final_gdb
mosaic_path = os.path.join(final_gdb, mosaic_name)

# Delete existing mosaic dataset if it exists
if arcpy.Exists(mosaic_path):
    print(f"🗑️ Deleting existing Mosaic Dataset: {mosaic_path}")
    arcpy.Delete_management(mosaic_path)

# Create mosaic dataset
print(f"📦 Creating Mosaic Dataset: {mosaic_path}")
arcpy.CreateMosaicDataset_management(
    final_gdb,
    mosaic_name,
    spatial_ref,
    product_definition="NONE",
)


# --- Add PRE rasters ---
print(f"➕ Adding PRE rasters to: {mosaic_name}")
arcpy.AddRastersToMosaicDataset_management(
    in_mosaic_dataset=mosaic_name,
    raster_type="Raster Dataset",
    input_path=base_dir,
    filter="*PRE*.tif",
    update_cellsize_ranges="UPDATE_CELL_SIZES",
    update_boundary="UPDATE_BOUNDARY",
    update_overviews="NO_OVERVIEWS",
    sub_folder="SUBFOLDERS",
    duplicate_items_action="ALLOW_DUPLICATES",
    build_pyramids="BUILD_PYRAMIDS",
    calculate_statistics="CALCULATE_STATISTICS",
    build_thumbnails="NO_THUMBNAILS",
    estimate_statistics="ESTIMATE_STATISTICS",
    enable_pixel_cache="NO_PIXEL_CACHE"
)

# Tag ProductName for PRE rasters
print(f"📝 Tagging ProductName as {product_name_PRE}")
with arcpy.da.UpdateCursor(mosaic_name, ["Category", "ProductName"]) as cursor:
    for row in cursor:
        if row[0] == 1:
            row[1] = product_name_PRE
            cursor.updateRow(row)

# --- Add PRE rasters ---
print(f"➕ Adding PRE rasters to: {mosaic_name}")
arcpy.AddRastersToMosaicDataset_management(
    in_mosaic_dataset=mosaic_name,
    raster_type="Raster Dataset",
    input_path=base_dir,
    filter="*PST*.tif",
    update_cellsize_ranges="UPDATE_CELL_SIZES",
    update_boundary="UPDATE_BOUNDARY",
    update_overviews="NO_OVERVIEWS",
    sub_folder="SUBFOLDERS",
    duplicate_items_action="ALLOW_DUPLICATES",
    build_pyramids="BUILD_PYRAMIDS",
    calculate_statistics="CALCULATE_STATISTICS",
    build_thumbnails="NO_THUMBNAILS",
    estimate_statistics="ESTIMATE_STATISTICS",
    enable_pixel_cache="NO_PIXEL_CACHE"
)

# Tag ProductName for PRE rasters
print(f"📝 Tagging ProductName as {product_name_PST}")
with arcpy.da.UpdateCursor(mosaic_name, ["Category", "ProductName"]) as cursor:
    for row in cursor:
        if row[0] == 1 and (row[1] is None or str(row[1]).strip() == ""):
            row[1] = product_name_PST
            cursor.updateRow(row)


📦 Creating Mosaic Dataset: \\WM20ocqu46ph01\WF_Projects\DK_THO\2_FINAL\CONSTR\CONSTR.gdb\DK_THO_AsBuilt_ArmourLayer_ra_UTM32N_v0
➕ Adding PRE rasters to: DK_THO_AsBuilt_ArmourLayer_ra_UTM32N_v0
📝 Tagging ProductName as DK_THO_AsBuilt_ArmourLayer_PreSurvey_ra_UTM32N_v0
➕ Adding PRE rasters to: DK_THO_AsBuilt_ArmourLayer_ra_UTM32N_v0
📝 Tagging ProductName as DK_THO_AsBuilt_ArmourLayer_PostSurvey_ra_UTM32N_v0


### 6)  AutoCAD Update


🛠️ AutoCAD Update Instructions
- Every new layer must include a **Layer** field in the attribute table to ensure compatibility between **ArcGIS Pro and AutoCAD**.  
- Layer names in the attribute table should **match the entries** in the **Excel log file** for traceability and consistency.

---
 🔹 Preparing the DWG for Export

1. **Open the DWG file** and delete existing layers that will be replaced.  
   - Use the AutoCAD command:  
     LAYDEL → Enter → Type N → Enter → Select layers by name
     
   - ⚠️ **Warning**: LAYDEL permanently deletes the selected layers and all objects on them. Use with caution.

   **Example layers to delete:**
   - ANNOTATION - As Built Footprint - MPI Adventure`
   - ANNOTATION - As Built Monopile Location`
   - VESSEL - As Built Footprint - MPI Adventure`
   - AS BUILT - Monopile Location`
   - AS BUILT - Monopile Outer Diameter 8pt6m`
   

2. **Save the DWG file** after cleanup.

---

 🔹 Exporting Updated Data from ArcGIS Pro

3. **Open the ArcGIS project MAP:* 005356185-{new version} THOR Master Construction Drawing CAD* that contains the new layers


4. Use the **Export to CAD** geoprocessing tool:
   - Ensure only required NEW layers are selected for export.Layers are new in ArcGIS Pro.
 

5. **Remove non-essential layers** that should not be included in the export:
   - For example:  `Bathymetry raster` (use contours instead of raster for CAD exports)

6. Use the **Append** option to add data to the existing DWG file without overwriting unrelated content.

7. Export the **.aprx** as a **Project Package**, ensuring that excluded `Bathymetry` are not part of the package.

#### 🗺️ Annotations: 
1:25000 scale when exporting WTG Layout annotation. below is the layer link for future reference:\\WM20ocqu46ph01\WF_Projects\DK_THO\0_ARCPRO\LAYER FILES\DK_THO_Layout_pt_UTM32N_v8.lyrx

#### CAD Tips:
1) 🔍 Zoom to Layer 
      - Type QSELECT → Enter, Set Property = Layer, and choose your layer --> OK 
      - Zoom to Selection: -Type Z → Enter--> Type O (for Object) → Enter



###  8) 🗺️ Export Map Package Workflow

1) Change in the code: `new_version = "XX"`) 
2) Run the code
3) Check output folder MCD_SCRATCH--> 01_ConsolidatedPackage and Open Map
4) Run tool`"Package Project"`-->  setting: Share outside of organization, History not included, All versions
      - ✅ Ensure *Bathy* and *MBES* layers are not included 
      - ✅ Check that data sources are correctly linked to the GDB (Geodatabase), etc.  


In [ ]:
import arcpy
import os
import shutil

new_version = 13

project_path = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\_MCD\005356185_THOR_Master_Construction_Drawing.aprx"
output_folder = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\_MCD\MCD_SCRATCH\01_ConsolidatedPackage"
temp_aprx_path = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\_MCD\TempProject.aprx"
map_name = f"005356185-{new_version} THOR Master Construction Drawing CAD"

if os.path.exists(temp_aprx_path):
    os.remove(temp_aprx_path)

shutil.copyfile(project_path, temp_aprx_path)

aprx = arcpy.mp.ArcGISProject(temp_aprx_path)

maps = aprx.listMaps(map_name)
if not maps:
    raise ValueError(f"Map '{map_name}' not found in project.")
map_obj = maps[0]


layers_to_exclude = [
    "Bathymetry Fugro 2023",
    "MBES Bathymetry (2023), mbMSL (Fugro 2023 UXO Survey - 004896869-01)",
    "Nearshore Bathymetry - 004896852-02-FUGG - THOR 2023 LIDAR SURVEY REPORT",
    "Hillshade",
    "Hillshade Nearshore"
]

for layer in map_obj.listLayers():
    try:
        if hasattr(layer, 'name') and layer.name in layers_to_exclude:
            print(f"🗑️ Removed layer: {layer.name}")
            map_obj.removeLayer(layer)
    except Exception as e:
        print(f"⚠️ Skipped a layer due to an error: {e}")

# Get the remaining map (should be only one now)
target_map = next((m for m in aprx.listMaps() if m.name == map_name), None)
if target_map is None:
    raise ValueError(f"Map '{map_name}' not found in the temp project.")

output = os.path.join(output_folder, map_name.replace(":", "_"))  # Clean name

arcpy.management.ConsolidateMap(
    in_map=map_obj,
    output_folder=output,
    convert_data="CONVERT",
    convert_arcsde_data="PRESERVE_ARCSDE",
    extent="DEFAULT",
    apply_extent_to_arcsde="ALL",
    preserve_sqlite="CONVERT_SQLITE",
    select_related_rows="KEEP_ALL_RELATED_ROWS"
)




### 8) 🌐 Portal Update Instructions

📁 Open ArcGIS Project
*\\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\_MCD\005356185_THOR_Master_Construction_Drawing.aprx*

🗺️ Open the Map: - 005356185 THOR Master Construction Drawing Portal

📂 Manage Versions: - Move the previous version to the Approved group. - Move the new version to the Under Review group

➕ Add New Layers: - Ensure all new or updated data layers are added to the map

♻️ Overwrite Existing Map: - Overwrite: 005356185 THOR Master Construction Drawing Portal

### 9) 🖨️ Export Maps to the PDFs

- **Copy and paste** previous layout in ArcGIS Pro project:  005356185_THOR_Master_Construction_Drawing.aprx
- 🆕 **Update title** with new revision number
- 📊 **Update table** elements as required
- Edit in the code:  
    - *text_element = layout.listElements("TEXT_ELEMENT", "Text 27 - Revision Box - 01 Checked Initials **8**")[0]*
- 💻 **Run the script** to export updated map books

In [ ]:
import arcpy
import PyPDF2

# Define the path to your ArcGIS Pro project
new_version = "11"
aprx_path = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\_MCD\005356185_THOR_Master_Construction_Drawing.aprx"

# Load the project
aprx = arcpy.mp.ArcGISProject(aprx_path)

# Specify the layout name
layout_name = f"005356185-{new_version} THOR Master Construction Drawing"

# Check if the specified layout exists
layout = None
for lyr in aprx.listLayouts():
    if lyr.name == layout_name:
        layout = lyr
        break

if layout is None:
    raise ValueError(f"Layout '{layout_name}' not found in the project.")

# Access the map series
map_series = layout.mapSeries

# Access the map frame to control layers
map_frame = layout.listElements("MAPFRAME_ELEMENT", "Main Map Map Frame")[0]
map_ = map_frame.map

# List all layers in the map to verify the layer name
layers = map_.listLayers()
print("Available layers:")
for layer in layers:
    print(layer.name)

# Access the map series layer "DDP"
ddp_layer = None
for layer in layers:
    if layer.name == "DDP":
        ddp_layer = layer
        break

if ddp_layer is None:
    raise ValueError("Layer 'DDP' not found in the map.")

# Define the output directory for the map books
output_dir = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\_MCD"

# Define the definition queries for each map book
definition_queries = ["SCOPE = 'EEC'", "SCOPE = 'EAC'", "SCOPE = 'FOU'", "SCOPE = 'EOS'"]

# Define the text to update for each map book
approved_by_texts = ["DS", "AM", "RC/JC/OW","NK"]

# Define the output file names for each map book
output_files = ["005356185-11 THOR Master Construction Drawing - EEC.pdf",
                "005356185-11 THOR Master Construction Drawing - EAC.pdf",
                "005356185-11 THOR Master Construction Drawing - FOU.pdf",
                "005356185-11 THOR Master Construction Drawing - EOS.pdf"]

# Layers to be turned on for the first page only
first_page_layers = ["World Imagery (Clarity)", "Nearshore Bathymetry - 004896852-02-FUGG - THOR 2023 LIDAR SURVEY REPORT", "Hillshade Nearshore"]

# Export the first map book with layers on for the first page
for i, definition_query in enumerate(definition_queries):
    # Set the definition query for the map series layer
    ddp_layer.definitionQuery = definition_query
    
    # Update the text element
    text_element = layout.listElements("TEXT_ELEMENT", "Text 27 - Revision Box - 01 Checked Initials 8")[0]
    text_element.text = approved_by_texts[i]
    
    if i == 0:
        # Turn on the specified layers for the first page of the first map book
        for layer in layers:
            if layer.name in first_page_layers:
                layer.visible = True
        
        # Export the first page of the first map book
        first_page_path = output_dir + f"\\{output_files[i].replace('.pdf', '_first_page.pdf')}"
        map_series.exportToPDF(first_page_path, multiple_files="PDF_SINGLE_FILE", resolution=100, page_range_type="RANGE", page_range_string="1")
        
        # Turn off the specified layers for the rest of the pages
        for layer in layers:
            if layer.name in first_page_layers:
                layer.visible = False
        
        # Export the rest of the pages of the first map book (pages 2 to 35)
        remaining_pages_path = output_dir + f"\\{output_files[i].replace('.pdf', '_remaining_pages.pdf')}"
        map_series.exportToPDF(remaining_pages_path, multiple_files="PDF_SINGLE_FILE", resolution=100, page_range_type="RANGE", page_range_string="2-35")
        
        # Merge the first page and the remaining pages into a single PDF
        merger = PyPDF2.PdfFileMerger()
        merger.append(first_page_path)
        merger.append(remaining_pages_path)
        merger.write(output_dir + f"\\{output_files[i]}")
        merger.close()
    else:
        # Ensure the specified layers are off for the other map books
        for layer in layers:
            if layer.name in first_page_layers:
                layer.visible = False
        
        # Export the entire map book
        map_series.exportToPDF(output_dir + f"\\{output_files[i]}", multiple_files="PDF_SINGLE_FILE", resolution=100)

print("Map books exported successfully with updated text elements and 100 DPI resolution!")

### XXX) Add ANY LAYER  to MCD

#### ✅ Update LAYER Field

In [ ]:
import arcpy

# === Path to feature class ===
gdb_path = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\2_FINAL\VESSELS\VESSELS.gdb"
name_FC = "DK_THO_JB115_JackUpBarge_pt_UTM32N_v0"
fc = f"{gdb_path}\\{name_FC}"

# === Add LAYER field if missing ===
if "Layer" not in [f.name for f in arcpy.ListFields(fc)]:
    arcpy.AddField_management(fc, "LAYER", "TEXT")
    print("🆕 Field 'LAYER' added.")
else:
    print("✅ Field 'LAYER' already exists.")

# === Layer mapping ===
layer_map = {
    "JUB Install":               "VESSEL - As Planned JUB Mooring Plan Installation", 
    "JUB Stand Off":             "VESSEL - As Planned JUB Mooring Plan Standoff",
    "Mooring wire Installation": "VESSEL - As Planned JUB Mooring Plan Installation",
    "Mooring wire Stand Off":    "VESSEL - As Planned JUB Mooring Plan Standoff",
}

# === Update LAYER field ===
with arcpy.da.UpdateCursor(fc, ["LAYER"]) as cursor:
    for row in cursor:
        original = row[0]
        row[0] = layer_map.get(row[0], row[0])
        if row[0] != original:
            print(f"🔁 LAYER updated: '{original}' ➝ '{row[0]}'")
        cursor.updateRow(row)

print("✅ Section 1 complete: LAYER field updated.")


#### ✅ Update SYMBOL Field

In [ ]:
gdb_path = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\2_FINAL\VESSELS\VESSELS.gdb"
name_FC = "DK_THO_JB115_JackUpBarge_pt_UTM32N_v0"
fc = f"{gdb_path}\\{name_FC}"

# === Add SYMBOL field if missing ===
if "SYMBOL" not in [f.name for f in arcpy.ListFields(fc)]:
    arcpy.AddField_management(fc, "SYMBOL", "TEXT")
    print("🆕 Field 'SYMBOL' added.")
else:
    print("✅ Field 'SYMBOL' already exists.")

# === Symbol mapping ===
symbol_map = {
    "VESSEL - As Planned JUB Mooring Plan Installation": "As Planned JUB Mooring Installation",
    "VESSEL - As Planned JUB Mooring Plan Standoff":     "As Planned JUB Mooring Standoff",
}

# === Update SYMBOL field from LAYER ===
with arcpy.da.UpdateCursor(fc, ["LAYER", "SYMBOL"]) as cursor:
    for row in cursor:
        original = row[1]
        row[1] = row[0]  # Default: SYMBOL = LAYER
        # row[1] = symbol_map.get(row[1], row[1])  # Override if match found
        #if row[1] != original:
        #    print(f"🎨 SYMBOL updated: '{original}' ➝ '{row[1]}'")
        cursor.updateRow(row)

print("✅ Section 2 complete: SYMBOL field updated.")


#### ✅ Delete a Field from a Feature Class

In [ ]:
gdb_path = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\2_FINAL\VESSELS\VESSELS.gdb"
name_FC = "DK_THO_JB115_JackUpBarge_pt_UTM32N_v0"
fc = f"{gdb_path}\\{name_FC}"

# Field name to delete
field_to_delete = "Entity"

# Get a list of field names
field_names = [field.name for field in arcpy.ListFields(fc)]

# Check if the field exists and delete it
if field_to_delete in field_names:
    arcpy.DeleteField_management(fc, field_to_delete)
    print(f"🗑️ Field '*{field_to_delete}*' deleted from *{fc}*.")
else:
    print(f"ℹ️ Field '*{field_to_delete}*' does not exist in *{fc}*.")